### Assignment 5: Genetic Algorithm
Jake Giguere <giguere@bu.edu>

In [6]:
import enum
import numpy as np

In [7]:
class cities(enum.IntEnum):
    Boston = 0
    London = 1
    Mumbai = 2
    Shanghai = 3

### Define one hot function 


In [8]:
def one_hot(city, max_cities = 4):
    one_hot = np.zeros(max_cities)
    one_hot[city] = 1
    return one_hot


In [9]:
def crossover_one_hot(parent1, parent2, crossover_point, max_cities=4):
    import numpy as np
    
    # Function to convert one-hot vector to city index
    def one_hot_to_index(one_hot_vector):
        return np.argmax(one_hot_vector)
    
    # Copy head segment from Parent 1
    child_route = parent1[:crossover_point]
    child_cities = set(one_hot_to_index(city) for city in child_route)
    
    # Append tail segment from Parent 2, skipping duplicates
    for city in parent2[crossover_point:]:
        city_idx = one_hot_to_index(city)
        if city_idx not in child_cities:
            child_route.append(city)
            child_cities.add(city_idx)
    
    # Ensure all cities are included
    total_cities = max_cities
    if len(child_cities) < total_cities:
        # Find missing cities
        missing_cities = set(range(total_cities)) - child_cities
        # Append missing cities in order of appearance in Parent 2
        for city in parent2:
            city_idx = one_hot_to_index(city)
            if city_idx in missing_cities:
                child_route.append(city)
                child_cities.add(city_idx)
                missing_cities.remove(city_idx)
                if not missing_cities:
                    break
    
    # Return to starting city
    if not np.array_equal(child_route[0], child_route[-1]):
        child_route.append(child_route[0])
    
    return child_route


Define Cities and Indicies 

In [10]:
city_names = ['Boston', 'London', 'Mumbai', 'Shanghai']
city_indices = {name: idx for idx, name in enumerate(city_names)}


In [11]:
boston = one_hot(city_indices['Boston'])
london = one_hot(city_indices['London'])
mumbai = one_hot(city_indices['Mumbai'])
shanghai = one_hot(city_indices['Shanghai'])


In [12]:
# Parent 1 Route: Boston → Mumbai → London → Shanghai → Boston
parent1 = [boston, mumbai, london, shanghai, boston]

# Parent 2 Route: Boston → London → Mumbai → Shanghai → Boston
parent2 = [boston, london, mumbai, shanghai, boston]


In [13]:
crossover_point = 1  # As specified in your example

# Generate child route
child_route = crossover_one_hot(parent1, parent2, crossover_point, max_cities=4)


In [14]:
def route_to_city_names(route):
    return [city_names[np.argmax(city)] for city in route]

# Print the child route in terms of city names
print("Child Route:", ' → '.join(route_to_city_names(child_route)))


Child Route: Boston → London → Mumbai → Shanghai → Boston


### Mutation 

In [ ]:
import numpy as np
import random

def mutate_route(route):
    # Ensure the route has enough cities to perform a mutation
    if len(route) <= 3:
        # Not enough cities to swap (excluding start/end city)
        return route.copy()
    
    # Exclude first and last indices if the route starts and ends at the same city
    start_idx = 1
    end_idx = len(route) - 1 if np.array_equal(route[0], route[-1]) else len(route)
    
    # Select two distinct indices for swapping
    idx_range = list(range(start_idx, end_idx))
    i, j = random.sample(idx_range, 2)
    
    # Perform the swap
    mutated_route = route.copy()
    mutated_route[i], mutated_route[j] = mutated_route[j], mutated_route[i]
    
    return mutated_route
